# Code ideas

install stuff with

    import Pkg
    Pkg.add("Plots")

In [2]:
include("VMC/includeall.jl")
using Random
using BenchmarkTools

## Initialization

In [15]:
rng = Random.MersenneTwister(0)
num = 10
dims = 3
particles = Particles(dims, num, rng, wf);

## Wavefunctions

Use seperate structs for seperate types of wavefunctions and use multiple dispatch to use the same set of function names for all wavefunctions.

In [5]:
HOshape = [1.0, 1.0, √8];

In [6]:
wf = SimpleGaussian(0.5, HOshape)

SimpleGaussian{StaticArrays.SVector{3, Float64}}(0.5, [1.0, 1.0, 2.8284271247461903], [1.0, 1.0, 8.000000000000002])

In [3]:
wf = Correlated(0.4, 0.0433, HOshape)

Correlated{StaticArrays.SVector{3, Float64}}(0.4, 0.0433, [1.0, 1.0, 1.0], [1.0, 1.0, 1.0])

In [20]:
old_pos = particles.positions[1].+0.5
@btime ratio(particles, 1, old_pos, wf)

  35.045 ns (1 allocation: 16 bytes)


1.4752186872890891

In [ ]:
QF(positions, 1, wf)

## Hamiltonians

Do the same thing as for wavefunctions.

In [ ]:
include("hamiltonians.jl")

In [8]:
ham = HarmonicOscillator(0.6, [1.0, 1.0, 1.0])

HarmonicOscillator{StaticArrays.SVector{3, Float64}}(0.36, [1.0, 1.0, 1.0])

In [13]:
@btime potential(particles, ham)

  33.199 ns (1 allocation: 16 bytes)


2.206457929853552

## Sampler

The sampler should be a mutable struct with any shape for the parameter derivative.

In [ ]:
include("sampler.jl")

In [ ]:
samples = Samples(wf)

In [ ]:
sample!(samples, positions, wf, ham)
samples

## Times

In [ ]:
# correlated, importance - further optimizations
5.068250 seconds (70.00 M allocations: 3.875 GiB, 19.14% gc time)

In [ ]:
# correlated, importance - switching to vectors of MVectors and writing out matrix operations
9.633650 seconds (200.00 M allocations: 16.690 GiB, 44.44% gc time)

In [ ]:
44.047754 seconds (2.63 G allocations: 60.169 GiB, 34.77% gc time, 0.00% compilation time)

In [ ]:
# correlated, importance
48.440489 seconds (2.83 G allocations: 63.164 GiB, 34.12% gc time, 0.00% compilation time)

In [ ]:
109.517566 seconds (4.11 G allocations: 197.784 GiB, 47.83% gc time, 0.66% compilation time)

In [ ]:
112.990543 seconds (4.22 G allocations: 206.275 GiB, 48.09% gc time, 0.67% compilation time)

In [ ]:
1e7/12

In [ ]:
importance
11.540818 seconds (310.89 M allocations: 12.711 GiB, 27.59% gc time, 0.00% compilation time)

In [ ]:
importance
12.427354 seconds (330.00 M allocations: 14.753 GiB, 31.70% gc time)

In [ ]:
metro
7.231149 seconds (190.27 M allocations: 6.125 GiB, 21.49% gc time, 1.00% compilation time)

In [ ]:
metro
7.739034 seconds (200.00 M allocations: 7.153 GiB, 24.25% gc time)

In [ ]:
HOshape = sa.@SVector [1.0, 1.0, 1.4]
wf = Correlated(0.4, 0.01, HOshape)

In [ ]:
wf2 = SimpleGaussian(0.4, HOshape)

In [ ]:
@time QF(positions, 1, wf2, temp_dim)

In [ ]:
@time QF(positions, 1, wf, temp_dim)

In [ ]:
3-element Vector{Float64}:
 -1.0778540847860392
 -1.3543631323118932
  1.5217131105308506

In [ ]:
0.000034 seconds (58 allocations: 3.641 KiB)

## SArray

The particles are a dim x num [Static Array](https://github.com/JuliaArrays/StaticArrays.jl). If there are more than 100 elements, i.e 34 particles in 3D or 100 particles in 1D, normal arrays should be used instead. This should be benchmarked. Small arrays will be the focus of the masters anyway.

The potential shape should maybe be an SVector.

In [ ]:
import StaticArrays as sa

sa.SA[1, 2, 3]

a = sa.SMatrix{3, 2, Float64}([[1, 2, 3] [4,5,6]])
sa.@SVector [1.0, 1.0, 1.4]

## Autograd

Maybe use [ReverseDiff](https://github.com/JuliaDiff/ReverseDiff.jl) or [ForwardDiff](https://github.com/JuliaDiff/ForwardDiff.jl)? Which is faster depends on the number of input and output parameters and the number of operations and whether they are matrix operations. Some benchmarking is required.

In [ ]:
import ForwardDiff

In [ ]:
f(x::Vector) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);

x = rand(5)

g = x -> ForwardDiff.gradient(f, x); # g = ∇f

g(x)

ForwardDiff.hessian(f, x)